In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

from utils import *

%load_ext autoreload
%autoreload 2

In [2]:
def parse_cfg(cfgfile):
    """ 
    Parse a config file that states the network architecture.
    Official Yolo v3 config can be found online and in ./cfg/yolov3.cfg
    
    Returns a list of dicts, each dict describing a layer in the network
    """
    blocks = []

    with open(cfgfile) as f:
        # read in all lines, except for empty ones. :-1 is to skip the \n charachter
        lines = [line[:-1] for line in f if (line[:-1] != '')]
        # FROM TUTORIAL:
        lines = [x for x in lines if x[0] != '#']              # get rid of comments
        lines = [x.rstrip().lstrip() for x in lines]           # get rid of fringe whitespaces
        
        # skip through first non block lines. Uncomment second statement to skip 'net' info
        while (lines[0][0] != '['): # or (lines[0][:2] == '[n'):
            lines = lines[1:]
        
        # while the file is not empty, parse a new block
        i = 0
        while i < len(lines):
            block = {}
            block['type'] = lines[i][1:-1].rstrip()
            i += 1
            while (i < len(lines)) and (lines[i][0] != "["):
                key, val = lines[i].split('=')
                block[key.rstrip()] = val.lstrip()             # rstrip() + lstrip() FROM TUTORIAL
                i +=1
            blocks.append(block)
    
    return blocks

In [ ]:
blocks = parse_cfg("./cfg/yolov3.cfg")

In [3]:
def create_modules(blocks):
    # init
    net_info = blocks[0]             # store the network meta info separately
    mod_list = nn.ModuleList()       # This list will contain all our layers
    in_features = 3                  # Previous layer's # output channels (3 for RGB)
    out_features = []                # keep track of each layers # output channels
    
    for idx, block in enumerate(blocks[1:]):
        if block['type'] == 'convolutional':
            mod_list.append(create_conv_layer(block, in_features, idx))
            out_features.append(int(block['filters']))
            in_features = out_features[-1]
            
        elif block['type'] == 'upsample':
            mod_list.append(create_upsample_layer(block, idx))
            out_features.append(out_features[-1])
        
        elif block['type'] == 'route':
            mod_list.append(create_route_layer(block, idx))
            out_feature = sum([out_features[int(i)] for i in block['layers'].split(',')])
            out_features.append(out_feature)
            in_features = out_features[-1]
        
        elif block['type'] == 'shortcut':
            mod_list.append(create_shortcut_layer(block, idx))
            out_features.append(out_features[-1])
        
        elif block['type'] == 'yolo':
            mod_list.append(create_detection_layer(block, idx))
            out_features.append(out_features[-1])
    
        else:
            raise ValueError('Block type note recognised/implemented: {}'.format(block['type']))
    
    return net_info, mod_list

In [ ]:
net_info, mod_list = create_modules(blocks)

In [61]:
class Darknet(nn.Module):
    def __init__(self, config_file='./cfg/yolov3.cfg'):
        super(Darknet, self).__init__()
        self.blocks = parse_cfg(config_file)
        self.net_info, self.module_list = create_modules(self.blocks)
        
    def forward(self, x, gpu_enabled=False):
        
        modules = self.blocks[1:]  # get rid of 'fake' net-layer
        outputs = {}               # store all activation outputs to be accessed by routing and shortcuts
        bboxes  = torch.empty(0)   # concatenate all bbox predictions to this initially empty tensor
        
        # process all modules in order
        for i, module in enumerate(modules):
            
            # convolutional or upsample layer
            if module['type'] in ['convolutional', 'upsample']:
                x = self.module_list[i](x)
                outputs[i] = x
            
            # shortcut layer
            elif module['type'] == 'shortcut':
                frm = i + int(module['from'])
                x.add_(outputs[frm])
                outputs[i] = x
            
            # route layer
            elif module['type'] == 'route':
                layers = [int(layer) for layer in module['layers'].split(',')]     # get layers as int in a list
                layers = [layer if (layer > 0) else i+layer for layer in layers]   # make all layers absolute
                if len(layers) == 1:
                    x = outputs[layers[0]]
                elif len(layers) == 2:
                    x = torch.cat((outputs[layers[0]], outputs[layers[1]]), dim=1) # concatenate layers if more than 1
                else:
                    ValueError('Routing with more than 2 ({}) layers not implemented'.format(layers))
                
                outputs[i] = x
            
            # detection layer
            elif module['type'] == 'yolo':
                input_size  = int(self.net_info['width'])
                anchors     = self.module_list[i][0].anchors
                num_classes = int(module['classes'])
                
                preds       = predict_transform(x, input_size, anchors, num_classes)
                bboxes      = torch.cat((bboxes, preds), dim=1)
            
            else:
                raise ValueError('Block type note recognised/implemented: {}'.format(block['type']))
                
        return bboxes
                

In [62]:
darknet = Darknet()

In [65]:
out = darknet(torch.randn(1, 3, 32*13, 32*13))
print(out.shape)
print(out.dtype)

/Users/anner/Code/miniconda3/envs/pytorch/lib/python3.6/site-packages/torch/nn/modules/upsampling.py:122: UserWarning: nn.Upsampling is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.Upsampling is deprecated. Use nn.functional.interpolate instead.")
/Users/anner/Code/miniconda3/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:1961: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


torch.Size([1, 10647, 85])
torch.float32


In [54]:
a = torch.empty(0)
print(a.shape)
b = torch.ones(1, 4) * 2
print(b.shape)
c = torch.ones(5, 4) * 3
print(c.shape)

ab = torch.cat((a, b), 0)
print(ab.shape)
ab = torch.cat((ab, c), 0)
print(ab.shape)
print(ab)

torch.Size([0])
torch.Size([1, 4])
torch.Size([5, 4])
torch.Size([1, 4])
torch.Size([6, 4])
tensor([[2., 2., 2., 2.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]])


In [ ]:
time1 = 0
time2 = 0

for _ in range(10000):
    tic    = time.time()
    pred   = predict_transform1(out, anchors=[1, 2, 3], num_classes=80)
    time1 += time.time()-tic
    
    tic    = time.time()
    pred   = predict_transform2(out, anchors=[1, 2, 3], num_classes=80)
    time2 += time.time()-tic
    
print('elapsed time for 1: {:.2e}'.format(time1))
print('elapsed time for 2: {:.2e}'.format(time2))
print(pred.shape)

In [ ]:
n = 84 #84, 87
print(blocks[n])
layers = [int(layer) for layer in blocks[n]['layers'].split(',')]
print(len(layers))
a = np.arange(9)
print(a[layers[0]])

In [ ]:
print(mod_list[82][0].anchors)
print(mod_list[94][0].anchors)
print(mod_list[106][0].anchors)

In [ ]:
anchorstring = '10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326'
maskstring = '0,1,2'
anchors = [anchor for anchor in anchorstring.split(', ')]
print(anchors)
mask    = [int(msk) for msk in maskstring.split(',')] #:  0,1,2
anchors = [list(map(int, anchors[i].split(','))) for i in mask]
print(anchors)
# anchors = [[anchors[i], anchors[i+1]] for i in range(0, len(anchors), 2)]
print(anchors)

In [ ]:
n = -12

print(blocks[n]['type'])
conv_block = blocks[n]
for key, value in conv_block.items():
    print(key, ': ', value)

    
print([int(i) for i in conv_block['layers'].split(',')])
# out_feature = sum([out_features[int(i)] for i in conv_block['layers'].split()])


In [ ]:
print(blocks[9])
print(len(blocks))
for i, item in enumerate(blocks[1:]):
    print(i, item['type'])

In [ ]:
print(x)

In [ ]:
x = np.arange(9).reshape(3,3)
# x = x.astype(np.uint8)

In [ ]:
print(x)

In [ ]:
x = torch.tensor(x)
print(x.dtype)
y = torch.ones(3, 3)
print(y)

In [ ]:
seq = nn.Sequential(
    nn.Linear(4, 5),
    nn.Linear(5, 6))




In [ ]:
seq[0].output

In [ ]:
print(torch.eye(10))

In [ ]:
y.add_(x.type(torch.float))
print(y)

In [ ]:
print(y.max())

In [ ]:
print(y.shape)
print(x.shape)

In [ ]:
print(y.matmul(x))